In [1]:
import pandas as pd
import numpy as np 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\maity\AppData\Roaming\nltk_data...


True

## Load the dataset as pandas

In [32]:
df_train_pos = pd.read_csv("twitter-datasets/train_pos.txt",sep="\t",names=['tweets'])
df_train_pos['label'] = 1
df_train_neg = pd.read_csv("twitter-datasets/train_neg.txt",sep="\t",names=['tweets'])
df_train_neg['label'] = 0
df_train = pd.concat([df_train_pos, df_train_neg])
print(df_train_pos.shape)
print(df_train_neg.shape)
print(df_train.shape)

(97902, 2)
(99068, 2)
(196970, 2)


In [33]:
df_train.head()

,tweets,label
0,<user> i dunno justin read my mention or not ....,1
1,"because your logic is so dumb , i won't even c...",1
2,<user> just put casper in a box ! looved the...,1
3,<user> <user> thanks sir > > don't trip lil ma...,1
4,visiting my brother tmr is the bestest birthda...,1


## Remove tags and urls

In [21]:
def remove_tags(df):
    df_cleaned = df.copy()
    df_cleaned['tweets'] = df_cleaned['tweets'].apply(lambda tweet: re.sub(r'<.*?>', '', tweet).strip())
    return df_cleaned

In [34]:
remove_tags(df_train)

,tweets,label
0,i dunno justin read my mention or not . only j...,1
1,"because your logic is so dumb , i won't even c...",1
2,just put casper in a box ! looved the battle ...,1
3,thanks sir > > don't trip lil mama ... just ke...,1
4,visiting my brother tmr is the bestest birthda...,1
...,...,...
99063,can't wait to fake tan tonight ! hate being pale,0
99064,darling i lost my internet connection .. and i...,0
99065,kanguru defender basic 4 gb usb 2.0 flash driv...,0
99066,rizan is sad now,0


In [26]:
# Example to show the use of strip()
s = "<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me "
# Add the strip() method to remove unecessary spaces in the tweet after removing tags and urls
tokens = word_tokenize(re.sub(r'<.*?>', '', s).strip())
stop_words = stopwords.words('english')
l = [token for token in tokens if token.lower() not in stop_words]
print(l)
ps = PorterStemmer()
l_after_stem = [ps.stem(token) for token in l]
print(l_after_stem)
wordnet_lemmatizer = WordNetLemmatizer()
l_after_lem = [wordnet_lemmatizer.lemmatize(token) for token in l]
print(l_after_lem)

['dunno', 'justin', 'read', 'mention', '.', 'justin', 'god', 'knows', ',', 'hope', 'follow']
['dunno', 'justin', 'read', 'mention', '.', 'justin', 'god', 'know', ',', 'hope', 'follow']
['dunno', 'justin', 'read', 'mention', '.', 'justin', 'god', 'know', ',', 'hope', 'follow']


## Removing stop words + stemming + lemmatization

In [47]:
def tokenize_and_preprocess(df, stop_words = False, stemming = False, lemmatization = False):
    df_cleaned = df.copy()
    df_cleaned['tokens'] = df_cleaned['tweets'].apply(lambda tweet: word_tokenize(tweet))
    # remove stop words
    if stop_words:
        stop_words = stopwords.words('english')
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [token for token in tokens if token.lower() not in stop_words])
    # stemming
    if stemming:
        ps = PorterStemmer()
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [ps.stem(token) for token in tokens])
    # lemmatization
    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [wordnet_lemmatizer.lemmatize(token) for token in tokens])
    # remove the tweets columns
    df_cleaned.drop(['tweets'], axis=1, inplace=True)
    df_cleaned = df_cleaned.reindex(columns=['tokens', 'label'])
    return df_cleaned

In [48]:
tokenize_and_preprocess(remove_tags(df_train))

,tokens,label
0,"[i, dunno, justin, read, my, mention, or, not,...",1
1,"[because, your, logic, is, so, dumb, ,, i, wo,...",1
2,"[just, put, casper, in, a, box, !, looved, the...",1
3,"[thanks, sir, >, >, do, n't, trip, lil, mama, ...",1
4,"[visiting, my, brother, tmr, is, the, bestest,...",1
...,...,...
99063,"[ca, n't, wait, to, fake, tan, tonight, !, hat...",0
99064,"[darling, i, lost, my, internet, connection, ....",0
99065,"[kanguru, defender, basic, 4, gb, usb, 2.0, fl...",0
99066,"[rizan, is, sad, now]",0
